# Module 2 - Deploy an app

## Objectives

* Learn about application Deployments.
* Deploy your first app on Kubernetes with kubectl.

## Kubernetes Deployments

Once you have a running Kubernetes cluster, you can deploy your containerized applications on top of it. To do so, you create a Kubernetes **Deployment** configuration. The Deployment instructs Kubernetes how to create and update instances of your application. Once you've created a Deployment, the Kubernetes master schedules the application instances included in that Deployment to run on individual Nodes in the cluster.

Once the application instances are created, a Kubernetes Deployment Controller continuously monitors those instances. If the Node hosting an instance goes down or is deleted, the Deployment controller replaces the instance with an instance on another Node in the cluster. **This provides a self-healing mechanism to address machine failure or maintenance.**

In a pre-orchestration world, installation scripts would often be used to start applications, but they did not allow recovery from machine failure. By both creating your application instances and keeping them running across Nodes, Kubernetes Deployments provide a fundamentally different approach to application management.

>A Deployment is responsible for creating and updating instances of your application

## Deploying your first app on Kubernetes

<img src="https://d33wubrfki0l68.cloudfront.net/152c845f25df8e69dd24dd7b0836a289747e258a/4a1d2/docs/tutorials/kubernetes-basics/public/images/module_02_first_app.svg"  width="600" title="test" alt="">

You can create and manage a Deployment by using the Kubernetes command line interface, **Kubectl**. Kubectl uses the Kubernetes API to interact with the cluster. In this module, you'll learn the most common Kubectl commands needed to create Deployments that run your applications on a Kubernetes cluster.

When you create a Deployment, you'll need to specify the container image for your application and the number of replicas that you want to run. You can change that information later by updating your Deployment; [Module 5](./WKSHP-K8S101-mod5.ipynb) and [Module 6](./WKSHP-K8S101-mod6.ipynb) of the workshop discuss how you can scale and update your Deployments.

>Applications need to be packaged into one of the supported container formats in order to be deployed on Kubernetes 

For your first Deployment, you'll use a hello-node application packaged in a Docker container that uses NGINX to echo back all the requests. (If you didn't already try creating a hello-node application and deploying it using a container, you can do that first by following the instructions from the [Hello Minikube tutorial](https://kubernetes.io/docs/tutorials/hello-minikube/)).

Now that you know what Deployments are, let's go and deploy our first app!

A Pod is the basic execution unit of a Kubernetes application. Each Pod represents a part of a workload that is running on your cluster. [Learn more about Pods](https://kubernetes.io/docs/concepts/workloads/pods/).

### Lab 2a - Deploy an App
#### Step 1 of 3

kubectl comes installed in the online terminal. Type kubectl in the terminal to see its usage. The common format of a kubectl command is: kubectl action resource. This performs the specified action (like create, describe) on the specified resource (like node, container). You can use `--help` after the command to get additional info about possible parameters (`kubectl get nodes --help`).

To view the nodes in the cluster, run the `kubectl get nodes` command:

In [ ]:
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
port=$(expr $(echo ${studentId} | tr -cd '[[:digit:]]') + 8000)

kubectl get nodes

Here we see the available nodes. Kubernetes will choose where to deploy our application based on Node available resources.

#### Step 2 of 3 

Let’s deploy our first app on Kubernetes with the `kubectl create deployment` command. We need to provide the deployment name and app image location (include the full repository url for images hosted outside Docker hub).


In [ ]:
kubectl create deployment kubernetes-bootcamp-${studentId} --image=gcr.io/google-samples/kubernetes-bootcamp:v1

Great! You just deployed your first application by creating a deployment. This performed a few things for you:

* searched for a suitable node where an instance of the application could be run
* scheduled the application to run on that Node
* configured the cluster to reschedule the instance on a new Node when needed

To list your deployments use the `get deployments` command:

In [ ]:
kubectl get deployments | grep $studentId

We see that there is one deployment running a single instance of your app. The instance is running inside a Docker container on your node.

#### Step 3 of 3 

Pods that are running inside Kubernetes are running on a private, isolated network. By default they are visible from other pods and services within the same kubernetes cluster, but not outside that network. When we use `kubectl`, we're interacting through an API endpoint to communicate with our application.

We will cover other options on how to expose your application outside the kubernetes cluster in [module 4](./WKSHP-K8S101-mod4.ipynb).

We still have a couple of options to connect to our pod. 

The first option is to run the command ` kubectl exec ` to execute a command on the pod:

In [ ]:
POD_NAME=$(kubectl get pods -o go-template --template '{% raw %}{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}{% endraw %}' | grep ${studentId} ) 
echo Name of the Pod: $POD_NAME
kubectl exec $POD_NAME -- curl -s http://localhost:8080

The other option is to use `kubectl` to create a port-forward that will forward communications into the cluster-wide, private network. The port-forward can be terminated by pressing control-C and won't show any output while its running.

We will open a terminal window (located in the Launcher tab) to run the proxy.

>If you don't have a Launcher tab, use File->New launcher from the menubar or simply click the "+" sign below the menubar.

` From the Terminal window ` run the following commands (you can use CTRL-C/CTRL-V to cut/paste)

```
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
port=$(expr $(echo $studentId | tr -cd '[[:digit:]]') + 8000)
POD_NAME=$(kubectl get pods -o go-template --template '{% raw %}{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}{% endraw %}' | grep $studentId) 
kubectl port-forward $POD_NAME $port:8080
```

We now have a connection between our host (the online terminal) and the Kubernetes cluster. The port-forward enables direct access to the pod from this notebook.

You can see connect to the application using the `curl` command:


In [ ]:
curl http://localhost:$port

This terminates module 2. Let's continue with [module 3, Explore your app](./4-WKSHP-K8S101-mod3.ipynb).
